In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from scipy import stats
from scipy.stats import norm, skew
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer, mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt

In [ ]:
#readfile

stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
holiday = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')

## Exploratory Data Analysis

## Oil

In [ ]:
oil.head()

In [ ]:
# #todatetime

# train.date= pd.to_datetime(train.date)
# oil.date = pd.to_datetime(oil.date)
# holiday.date = pd.to_datetime(holiday.date)

In [ ]:
oil.head()

In [ ]:
import matplotlib.pyplot as plt

# Simpan data sebelum pengisian dan interpolasi
oil_before = oil['dcoilwtico'].copy()

# Langkah 1: Isi nilai kosong awal dengan 93.21
oil['dcoilwtico'].fillna(value=93.21)

# Langkah 2: Interpolasi nilai kosong lainnya
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate(method='linear')

# Simpan data setelah pengisian dan interpolasi
oil_after = oil['dcoilwtico']

# Membuat grafik
plt.figure(figsize=(14, 6))

# Plot sebelum pengisian
plt.subplot(1, 2, 1)
plt.plot(oil_before, marker='o', linestyle='-', markersize=4, lw=1, label='Before Fill and Interpolation', color='blue')
plt.title('Before Fill and Interpolation')
plt.xlabel('Index')
plt.ylabel('dcoilwtico')
plt.legend()

# Plot setelah pengisian
plt.subplot(1, 2, 2)
plt.plot(oil_after, marker='o', linestyle='-', markersize=4, lw=1, label='After Fill and Interpolation', color='orange')
plt.title('After Fill and Interpolation')
plt.xlabel('Index')
plt.ylabel('dcoilwtico')
plt.legend()

# Menampilkan grafik
plt.tight_layout()
plt.show()


In [ ]:
# oil['dcoilwtico'].dropna()

In [ ]:
# oil['dcoilwtico'].fillna(value=93.21)

In [ ]:
# oil=oil.dropna()
# oil.isna().sum()

In [ ]:
oil.head()

In [ ]:
# Langkah 1: Isi nilai kosong awal dengan 93.21
oil['dcoilwtico'].fillna(value=93.21, inplace=True)

# Langkah 2: Interpolasi nilai kosong lainnya
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate(method='linear')

# Tampilkan hasil
oil.head()


In [ ]:
oil.info()

In [ ]:
oil.head()

In [ ]:
oil.isnull().sum()

In [ ]:
oil["date"]=pd.to_datetime(oil["date"])
oil['year'] = oil['date'].dt.year
oil['month'] = oil['date'].dt.month
oil['dayofweek'] = oil['date'].dt.dayofweek
oil['day'] = oil['date'].dt.day

In [ ]:
oil.head()

In [ ]:
# Moving average 7 hari
oil['moving_avg_7'] = oil['dcoilwtico'].rolling(window=7).mean()

# Moving average 30 hari
oil['moving_avg_30'] = oil['dcoilwtico'].rolling(window=30).mean()

# Moving average 90 hari
oil['moving_avg_90'] = oil['dcoilwtico'].rolling(window=90).mean()

# Tampilkan beberapa baris pertama untuk melihat hasilnya
oil.head(10)

In [ ]:
# Plot the moving averages
plt.figure(figsize=(10, 6))

# Plot original data (dcoilwtico)
plt.plot(oil['date'], oil['dcoilwtico'], label='Oil Price', color='blue', linewidth=2)

# Plot moving averages
plt.plot(oil['date'], oil['moving_avg_7'], label='7-day Moving Average', color='green', linestyle='--', linewidth=2)
plt.plot(oil['date'], oil['moving_avg_30'], label='30-day Moving Average', color='orange', linestyle='--', linewidth=2)
plt.plot(oil['date'], oil['moving_avg_90'], label='90-day Moving Average', color='red', linestyle='--', linewidth=2)

# Add title and labels
plt.title('Oil Price and Moving Averages', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)

# Add legend
plt.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
oil['date'] = pd.to_datetime(oil['date'])

# Membuat kolom baru yang menunjukkan minggu dalam satu tahun
oil['week_of_year'] = oil['date'].dt.isocalendar().week

In [ ]:
oil.head(10)

## Train

In [ ]:
# Group by family and calculate total sales
sales_by_family = train.groupby('family').sales.sum()

# Plot
plt.figure(figsize=(10, 6))
sales_by_family.plot(kind='bar')
plt.title('Sales by Family')
plt.xlabel('Family')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
train.groupby(['store_nbr','family']).sales.sum()

In [ ]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['dayofweek'] = train['date'].dt.isocalendar().week
train['day'] = train['date'].dt.day   

In [ ]:
train['dayname'] = train.date.dt.strftime('%A')

In [ ]:
day_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [ ]:
train.groupby('dayname').sales.mean().reindex(index=day_order).plot(kind='bar')
plt.title('sales of week')

In [ ]:
#calculate total sales by family

sales_by_store_nbr = train.groupby('store_nbr').sales.sum()

# Plot

plt.figure(figsize=(10, 6))
sales_by_store_nbr.plot(kind='bar')
plt.title('Total Sales by Store_number')
plt.xlabel('Store_nbr')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
# train1.head()

In [ ]:
# train1 =train.drop(columns = ['id','onpromotion'])
# train1['date'] = train1.date.dt.to_period('D')
# store_sales = train1.set_index(['store_nbr', 'family', 'date']).sort_index()
# average_sales = (
#     store_sales
#     .groupby('date').mean()
#     .squeeze()
# )

# X = average_sales
# X["week"] = X.index.week
# X["day"] = X.index.dayofweek

# seasonal_plot(X, y='sales', period='week', freq='day');

In [ ]:
# print(freqencies.shape)
# print(spectrum.shape)

In [ ]:
# #Importing Libraries
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from learntools.time_series.utils import plot_periodogram, seasonal_plot
# from statsmodels.tsa.deterministic import DeterministicProcess
# from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

# import warnings
# warnings.filterwarnings("ignore")
# plot_periodogram(average_sales);

## Feature Engineering

In [ ]:
train.head()

In [ ]:
#define average sales per faimly

family_mean = train.groupby("family").sales.mean().reset_index()
family_mean.columns = ["family", "family_mean"]
family_mean.head()

In [ ]:
#average per store

store_nbr_mean = train.groupby("store_nbr").sales.mean().reset_index()
store_nbr_mean.columns = ["store_nbr", "store_nbr_mean"]
store_nbr_mean.head()

In [ ]:
train['nbr_of_weeks'] = train['date'].dt.isocalendar().week

In [ ]:
train['nbr_of_days'] = train.date.dt.day

In [ ]:
# calculate total sales by group
sales_by_month = train.groupby('nbr_of_days').sales.mean()

# Plot
plt.figure(figsize=(10, 6))
sales_by_month.plot(kind='line')
plt.title('Total Sales by number of days')
plt.xlabel('days_nbr')
plt.ylabel('Mean Sales')
plt.show()

In [ ]:
# Menghitung total penjualan per minggu
sales_by_week = train.groupby('nbr_of_weeks').sales.mean()  # Menggunakan mean untuk mendapatkan rata-rata penjualan per minggu

# Membuat grafik
plt.figure(figsize=(10, 6))
sales_by_week.plot(kind='line')
plt.title('Total Sales by Number of Weeks')
plt.xlabel('Week Number')
plt.ylabel('Mean Sales')
plt.show()

In [ ]:
#average sales per weeks

weaks_mean = train.groupby("nbr_of_weeks").sales.mean().reset_index()
weaks_mean.columns = ["nbr_of_weeks", "weaks_mean"]
weaks_mean 

In [ ]:
#average sales per month

month_mean = train.groupby("nbr_of_days").sales.mean().reset_index()
month_mean.columns = ["nbr_of_days", "month_mean"]
month_mean.head(20)

In [ ]:
import plotly.express as px

promotion_sales = train.groupby('onpromotion')['sales'].sum().reset_index()

fig = px.bar(promotion_sales, x='onpromotion', y='sales',
             labels={'onpromotion': 'On Promotion', 'sales': 'Total Sales'},
             title='Relationship between On Promotion and Total Sales',
             width=800, height=600)

fig.update_xaxes(range=[-0.5, 1.5])

fig.show()

fig_hist = px.histogram(train, x='sales', nbins=50,
                        labels={'sales': 'Sales'},
                        title='Distribution of Sales',
                        width=800, height=600)

fig_hist.show()


In [ ]:

promotion_sales_by_family = train.groupby('family').agg({'onpromotion': 'sum', 'sales': 'sum'}).reset_index()

fig = px.scatter(promotion_sales_by_family, x='onpromotion', y='sales',
                 labels={'onpromotion': 'Total Promotions', 'sales': 'Total Sales'},
                 title='Relationship between Total Promotions and Total Sales per Product Family',
                 width=800, height=600)

# Show the plot
fig.show()


In [ ]:
promotion_sales = train.groupby('onpromotion')['sales'].sum().reset_index()

total_promotion_sales = promotion_sales[promotion_sales['onpromotion'] == True]['sales'].values[0]

total_no_promotion_sales = promotion_sales[promotion_sales['onpromotion'] == False]['sales'].values[0]

promotion_ratio = total_promotion_sales / (total_promotion_sales + total_no_promotion_sales)

train['is_promotion'] = train['onpromotion'].apply(lambda x: promotion_ratio if x else 1 - promotion_ratio)

## Merge Data and Prepare to Modelling

In [ ]:
#mergee

train = pd.merge(train, family_mean, how = 'left', on = 'family')

In [ ]:
# union_final = train.merge(oil,on=["year","month","day"],how="left")

In [ ]:
# union_final.info()

In [ ]:
# union_final=union_final.drop(axis=1,columns=["dayofweek_x","date_y","dayofweek_y"])
# union_final=pd.get_dummies(data=union_final,columns=["family"],drop_first=True)
# union_final=union_final.dropna()
# x_final=union_final.drop(axis=1,columns=["id","date_x","sales"])
# y_final=union_final["sales"]
# print(x_final.shape)
# print(y_final.shape)
# xtrain_final,xtest_final,ytrain_final,ytest_final=train_test_split(x_final,y_final,test_size=0.2)

In [ ]:
# print(xtrain_final.shape)
# print(ytrain_final.shape)
# print(xtest_final.shape)
# print(ytest_final.shape)

In [ ]:
# from sklearn.metrics import r2_score
# final_forest=RandomForestRegressor(max_depth=None, min_samples_leaf= 2, min_samples_split= 2, n_estimators=225)
# final_forest.fit(xtrain_final,ytrain_final)
# prediction=final_forest.predict(xtest_final)
# Final_r2_score=r2_score(ytest_final,prediction)
# print("Final R2 score =",Final_r2_score)

In [ ]:
# from sklearn.metrics import r2_score
# Final_r2_score=r2_score(ytest_final,prediction)
# print("Final R2 score =",Final_r2_score)

In [ ]:
# test["date"]=pd.to_datetime(test["date"])
# test['year'] = test['date'].dt.year
# test['month'] = test['date'].dt.month
# test['day'] = test['date'].dt.day
# test['dayofweek'] = test['date'].dt.dayofweek

In [ ]:
# test.head()

In [ ]:
# test_union=test.merge(oil,on=["year","month","day"],how="left")

In [ ]:
# test_union=test_union.drop(axis=1,columns=["dayofweek_x","date_y","dayofweek_y"])
# test_union=pd.get_dummies(data=test_union,columns=["family"],drop_first=True)
# test_union["dcoilwtico"].interpolate(inplace=True)
# test_union.isna().sum()

In [ ]:
# test_union.head()

In [ ]:
# union_final.info()

In [ ]:
# test_union.info()

In [ ]:
# x_final.info()

In [ ]:
# x_test.info()

In [ ]:
# x_test = x_test.reindex(columns=x_final.columns, fill_value=0)

In [ ]:
# x_test = x_test.sort_values(by=['day',  'month','year' ], ascending=[True, True, True])

In [ ]:
# x_test.head()

In [ ]:
# x_test = x_test.sort_values(by=['day','year','month'], ascending=[True, True, True])

In [ ]:
# # Sort by 'year', 'month', and 'day' in ascending order
# x_test_sort = x_test.sort_values(by=['day','year','month'], ascending=[True, True, True])

# # Display the sorted DataFrame
# x_test_sort.head()


In [ ]:
# x_test.info()

In [ ]:
# x_test=test_union.drop(axis=1,columns=["id","date_x"])
# Final_predictions=final_forest.predict(x_test)
# output = pd.DataFrame({'id': test_union['id'], 'sales': Final_predictions})
# output.to_csv('Gass.csv', index=False)

# print("Done")

In [ ]:
oil.head()

In [ ]:
train = pd.merge(train, store_nbr_mean, how = 'left', on = 'store_nbr')
train = pd.merge(train, month_mean, how = 'left', on = 'nbr_of_days')
train = pd.merge(train, weaks_mean, how = 'left', on = 'nbr_of_weeks')

In [ ]:
train = pd.merge(train, oil, how='left', on='date')
train.head()

In [ ]:
train.info()

## Modelling

In [ ]:
train.fillna(0, inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
#random forestedd

cols = ["store_nbr_mean", "onpromotion", "dcoilwtico","is_promotion" ,"family_mean","month_mean", "weaks_mean", "moving_avg_7", "moving_avg_30", "moving_avg_90", "day", "week_of_year", "month"] 
X = train[cols]
y = train['sales']

In [ ]:
x_train, x_test, t_train, t_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Fit the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, t_train)

# Get feature importances
importances = model.feature_importances_

# Create a DataFrame to display the feature importance
feature_importance_df = pd.DataFrame({
    'Feature': x_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Visualize the importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel("Feature Importance")
plt.title("Feature Importance in Random Forest")
plt.show()


In [ ]:
model = RandomForestRegressor(n_estimators = 120, n_jobs = -1, random_state=77)
model.fit(x_train, t_train)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_log_error
rmsle = np.sqrt(mean_squared_log_error(t_test, predictions))

In [ ]:
print(f"RMSLE: {rmsle:.4f}") 

In [ ]:
# import lightgbm as lgb

# # Inisiasi model LightGBM
# lgb_model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.01, max_depth=10)

# # Melatih model
# lgb_model.fit(x_train, t_train)

# # Prediksi dengan LightGBM
# predictions = lgb_model.predict(x_test)
# predictions = np.clip(predictions, a_min=0, a_max=None)

# # Hitung RMSLE
# rmsle = np.sqrt(mean_squared_log_error(t_test, predictions))
# print(f"RMSLE: {rmsle:.4f}")

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
# from sklearn.metrics import mean_squared_log_error
# import lightgbm as lgb

# # Data Preparation (sesuaikan dengan dataset Anda)
# # Pastikan train sudah ada dan data preprocessed
# cols = ["store_nbr_mean", "onpromotion", "dcoilwtico", "is_promotion", "family_mean", 
#         "month_mean", "weaks_mean", "moving_avg_7", "moving_avg_30", "moving_avg_90", "day", "week"]
# X = train[cols]
# y = train['sales']

# # Split data into training and testing
# x_train, x_test, t_train, t_test = train_test_split(X, y, test_size=0.2, random_state=77)

# ### Step 1: Log Transform Target for RMSLE Improvement ###
# t_train_log = np.log1p(t_train)  # Log-transform target
# t_test_log = np.log1p(t_test)    # Log-transform test target for evaluation

# ### Step 2: Hyperparameter Tuning with GridSearchCV ###
# # Define the parameter grid
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'max_features': ['auto', 'sqrt', 'log2']
# }

# # Initialize the RandomForest model
# model = RandomForestRegressor(random_state=77, n_jobs=-1)

# # Perform GridSearchCV to find the best hyperparameters
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_log_error', cv=3, n_jobs=-1, verbose=2)
# grid_search.fit(x_train, t_train_log)  # Use log-transformed target for training

# # Get the best parameters
# best_params = grid_search.best_params_
# print(f"Best Parameters: {best_params}")

# # Train the final model with the best parameters
# best_model = RandomForestRegressor(**best_params, random_state=77, n_jobs=-1)
# best_model.fit(x_train, t_train_log)  # Train with log-transformed target

# ### Step 3: Make Predictions and Inverse Log Transform ###
# predictions_log = best_model.predict(x_test)  # Predict on log-transformed target
# predictions = np.expm1(predictions_log)       # Convert back to the original scale

# # Clip predictions to avoid negative values
# predictions = np.clip(predictions, a_min=0, a_max=None)

# # Evaluate the model using RMSLE
# rmsle = np.sqrt(mean_squared_log_error(t_test, predictions))
# print(f"RMSLE: {rmsle:.4f}")

# ### Step 4: Cross-Validation to Get Consistent Scores ###
# # Perform Cross-Validation with the final model
# cv_scores = cross_val_score(best_model, x_train, t_train_log, scoring='neg_mean_squared_log_error', cv=5)
# rmsle_cv = np.sqrt(-cv_scores)

# print(f"Mean RMSLE (CV): {rmsle_cv.mean()}, Std RMSLE (CV): {rmsle_cv.std()}")

# ### Optional: Try LightGBM Model for Comparison ###
# # Initialize LightGBM model
# lgb_model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.01, max_depth=10)

# # Train the LightGBM model on log-transformed target
# lgb_model.fit(x_train, t_train_log)

# # Make predictions with LightGBM
# lgb_predictions_log = lgb_model.predict(x_test)
# lgb_predictions = np.expm1(lgb_predictions_log)  # Convert back to original scale

# # Clip predictions to avoid negative values
# lgb_predictions = np.clip(lgb_predictions, a_min=0, a_max=None)

# # Evaluate LightGBM model using RMSLE
# lgb_rmsle = np.sqrt(mean_squared_log_error(t_test, lgb_predictions))
# print(f"LightGBM RMSLE: {lgb_rmsle:.4f}")


## XGBoost Model

In [ ]:
# train_y = np.log1p(train['sales'])



# train.drop(columns=['sales'], inplace=True)
# Merge_data = pd.concat([train, test], keys=['train', 'test'])

# numeric_features = Merge_data.select_dtypes(include=['int64', 'float64']).columns
# categorical_features = Merge_data.select_dtypes(include=['object']).columns

# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])


# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

In [ ]:
# model = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', xgb.XGBRegressor(objective='reg:squarederror', n_estimators=11500, learning_rate=0.05, max_depth=9, subsample=0.9, colsample_bytree=0.9))])
# train_data = Merge_data.xs('train')
# test_data = Merge_data.xs('test')
# model.fit(train_data, train_y )

In [ ]:
# rmse = np.sqrt(-cross_val_score(model, train_data, train_y , scoring='neg_mean_squared_error', cv=5))
# print("RMSE: {:.4f} ({:.4f})".format(rmse.mean(), rmse.std()))

In [ ]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_log_error
# # Split data (menggunakan split yang sama seperti sebelumnya)
# x_train, x_test, t_train, t_test = train_test_split(X, y, test_size=0.2, random_state=77)

# # Inisiasi model XGBoost
# xgb_model = xgb.XGBRegressor(
#     objective='reg:squarederror',
#     n_estimators=1000,
#     learning_rate=0.01,
#     max_depth=10,
#     min_child_weight=1,
#     gamma=0.1,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     reg_alpha=0.1,
#     reg_lambda=1,
#     random_state=42,
#     tree_method='gpu_hist'
# )

# # Melatih model XGBoost
# xgb_model.fit(x_train, t_train)

# # Evaluasi model
# predictions = xgb_model.predict(x_test)
# # Clip negative predictions to 0
# predictions = np.clip(predictions, a_min=0, a_max=None)

# rmsle = np.sqrt(mean_squared_log_error(t_test, predictions))
# print(f"RMSLE: {rmsle:.4f}") 

In [ ]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout
# from sklearn.preprocessing import MinMaxScaler

# # Scaling the data (karena LSTM sensitif terhadap skala data)
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_train = scaler.fit_transform(x_train)
# scaled_test = scaler.transform(x_test)

# # Membuat model LSTM
# lstm_model = Sequential()
# lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(scaled_train.shape[1], 1)))
# lstm_model.add(Dropout(0.2))
# lstm_model.add(LSTM(units=50, return_sequences=False))
# lstm_model.add(Dropout(0.2))
# lstm_model.add(Dense(units=1))

# # Compile model
# lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# # Melatih model LSTM
# lstm_model.fit(scaled_train, t_train, epochs=5, batch_size=32)

# # Prediksi menggunakan LSTM
# lstm_predictions = lstm_model.predict(scaled_test)
# rmsle = np.sqrt(mean_squared_log_error(scaled_test, lstm_predictions))
# print(f"RMSLE: {rmsle:.4f}") 

## Evaluation

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_log_error
# import numpy as np
# import pandas as pd

# # Pilih fitur yang relevan
# cols = ["store_nbr_mean", "onpromotion", "dcoilwtico", "is_promotion", "family_mean", 
#         "month_mean", "weaks_mean", "moving_avg_7", "moving_avg_30", "moving_avg_90"]
# X = train[cols]
# y = train['sales']

# # Split data
# x_train, x_test, t_train, t_test = train_test_split(X, y, test_size=0.2, random_state=77)

# # Inisiasi model RandomForest dengan parameter yang dioptimalkan
# model = RandomForestRegressor(
#     n_estimators=300,   # Lebih banyak estimators untuk performa lebih stabil
#     max_depth=15,       # Membatasi kedalaman pohon
#     min_samples_split=5,  # Minimum sampel untuk memisahkan node
#     max_features='sqrt',  # Menggunakan akar kuadrat dari fitur
#     random_state=77,
#     n_jobs=-1  # Memaksimalkan penggunaan CPU
# )

# # Melatih model
# model.fit(x_train, t_train)

# # Evaluasi model
# predictions = model.predict(x_test)

# # Menggunakan clip untuk mencegah prediksi negatif
# predictions = np.clip(predictions, a_min=0, a_max=None)

# # Hitung RMSLE
# rmsle = np.sqrt(mean_squared_log_error(t_test, predictions))
# print(f"RMSLE: {rmsle:.4f}")


In [ ]:
# from sklearn.model_selection import GridSearchCV

# # Mengatur parameter yang akan diuji
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, None],
#     'min_samples_split': [2, 5, 10],
#     'max_features': ['auto', 'sqrt', 'log2']
# }

# # Membuat model RandomForestRegressor
# model = RandomForestRegressor(random_state=77, n_jobs=-1)

# # Menggunakan GridSearchCV untuk hyperparameter tuning
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_log_error', cv=3, n_jobs=-1, verbose=2)

# # Latih model menggunakan GridSearch
# grid_search.fit(x_train, t_train)

# # Mendapatkan parameter terbaik
# best_params = grid_search.best_params_
# print(f"Best Parameters: {best_params}")

# # Latih ulang model dengan parameter terbaik
# best_model = RandomForestRegressor(**best_params, random_state=77, n_jobs=-1)
# best_model.fit(x_train, t_train)

In [ ]:
# from sklearn.metrics import make_scorer, mean_squared_log_error
# predictions1 = best_model.predict(x_test)
# rmsle = np.sqrt(mean_squared_log_error(t_test, predictions))

In [ ]:
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_log_error
# import numpy as np

# # Split data (menggunakan split yang sama seperti sebelumnya)
# x_train, x_test, t_train, t_test = train_test_split(X, y, test_size=0.2, random_state=77)

# # Inisiasi model LGBM
# lgb_model = lgb.LGBMRegressor(
#     objective='regression',
#     n_estimators=1000,
#     learning_rate=0.01,
#     max_depth=10,
#     min_child_weight=1,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     reg_alpha=0.1,
#     reg_lambda=1,
#     random_state=42,
#     device='gpu'  # Menjalankan di GPU jika tersedia
# )

# # Melatih model LGBM
# lgb_model.fit(x_train, t_train)

# # Evaluasi model
# predictions = lgb_model.predict(x_test)
# # Clip negative predictions to 0
# predictions = np.clip(predictions, a_min=0, a_max=None)

# rmsle = np.sqrt(mean_squared_log_error(t_test, predictions))
# print(f"RMSLE: {rmsle:.4f}")


## Testing

In [ ]:
data_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")

In [ ]:
data_test.date = pd.to_datetime(data_test.date)
data_test['day'] = data_test['date'].dt.day
data_test['week'] = data_test['date'].dt.isocalendar().week
data_test['month'] = data_test['date'].dt.month   
data_test['is_promotion'] = data_test['onpromotion'].apply(lambda x: promotion_ratio if x else 1 - promotion_ratio)
data_test = pd.merge(data_test, family_mean, how = 'left', on = 'family')
data_test = pd.merge(data_test, store_nbr_mean, how = 'left', on = 'store_nbr')
data_test['nbr_of_days'] = data_test.date.dt.day
data_test['nbr_of_weeks'] = data_test['date'].dt.isocalendar().week
data_test = pd.merge(data_test, month_mean, how = 'left', on = 'nbr_of_days')
data_test = pd.merge(data_test, weaks_mean, how = 'left', on = 'nbr_of_weeks')
data_test = pd.merge(data_test, oil, how='left', on='date')
data_test.head()

In [ ]:
data_test.fillna(0, inplace=True)

In [ ]:
data_test.isnull().sum()

In [ ]:
X_test = data_test[cols]
predictions_t = model.predict(X_test)

In [ ]:
predictions_t

In [ ]:
df_sub = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv', index_col='id')
df_sub.sales = predictions_t
df_sub.to_csv('./submission.csv')
test_df2 = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv') 
test_df2.head()